In [1]:
import json
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import random

import paddle
from paddle.io import Dataset, DataLoader
from paddle import nn
import paddle.nn.functional as F

import os
import pickle
from tqdm import tqdm

In [2]:
if paddle.device.is_compiled_with_cuda():
    paddle.device.set_device('gpu')
else:
    paddle.device.set_device('cpu')
print(paddle.device.get_device())
def seed_paddle(seed):
    seed = int(seed)
   
    random.seed(seed) 

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) 
    paddle.seed(seed)

seed_paddle(seed=1024)

gpu:0


In [3]:
size = 12
seq_len = size * size

In [4]:
class TsDataset(Dataset):
    def __init__(self, df):

        self.seq_list1 = list(df['Wspd_seq'])
        self.seq_list2 = list(df['Patv_seq'])
        
        self.seq_list3 = list(df['Pab1_seq'])
        self.seq_list4 = list(df['Temp_seq'])

        self.label_list = df.target.values

    def __getitem__(self, index):
        
        seq = self.seq_list1[index][-seq_len:]+self.seq_list2[index][-seq_len:]+\
                  self.seq_list3[index][-seq_len:]+self.seq_list4[index][-seq_len:]
        seq = np.array(seq).astype('float') 
        seq.resize(size, size, 4)

        label = np.array( self.label_list[index] ).astype( 'int' )       
        seq = paddle.to_tensor(seq)        

        return seq, label


    def __len__(self):
        return len(self.seq_list1)

In [5]:
class TSModel(nn.Layer):
    def __init__(self):
        super(TSModel, self).__init__()

        self.cnnLayer1 = nn.Sequential(
            nn.Conv2D(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2D(64),
            nn.GELU(),
            nn.AdaptiveAvgPool2D((12, 12))
        )
        self.res1 = nn.Conv2D(4, 64, kernel_size=1, stride=1)
        self.cnnLayer2 = nn.Sequential(
            nn.Conv2D(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2D(128),
            nn.GELU(),
        )
        self.res2 = nn.Conv2D(64, 128, kernel_size=1, stride=1)
        self.cnnLayer3 = nn.Sequential(
            nn.Conv2D(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2D(256),
            nn.GELU(),
            nn.AdaptiveAvgPool2D((3, 3))
        )

        self.Linear1 = nn.Linear(8, 8, bias_attr=True)
        self.Linear2 = nn.Linear(8, 1, bias_attr=True)

    def forward(self, X):

        cnn_out1 = self.cnnLayer1(X)
        res_out1 = self.res1(X)
        output1 = cnn_out1 + res_out1


        cnn_out2 = self.cnnLayer2(output1)
        res_out2 = self.res2(output1)
        output2 = cnn_out2 + res_out2

        cnn_out3 = self.cnnLayer3(output2)
        output3 = paddle.reshape(cnn_out3, ( cnn_out3.shape[0], 288, -1))

        out1 = self.Linear1(output3)
        final_output = self.Linear2(out1)


        return final_output

In [6]:
model = TSModel()
optimizer = paddle.optimizer.Adam(learning_rate=0.0005, parameters=model.parameters())
criterion = nn.MSELoss()

W0726 01:11:40.164940  1564 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0726 01:11:40.169754  1564 gpu_resources.cc:91] device: 0, cuDNN Version: 7.6.


In [7]:
df_train = pd.read_csv('../data/train_data_normal.csv')

In [8]:
len(df_train)

43500

In [9]:
cols = [x for x in df_train.columns if 'seq' in x or x=='target' or x=='Patv_space']
for col in cols:
    df_train[col] = df_train[col].apply(lambda x: json.loads(x))  
def get_diff(row):
    diff = np.array(row['Etmp_seq'])-np.array(row['Itmp_seq'])
    return list(diff)

df_train['Temp_seq'] = df_train.apply(get_diff, axis=1)

In [10]:
train_dataset = TsDataset(df_train)

train_loader = DataLoader(train_dataset,
                      batch_size=128,
                      shuffle=True,
                      num_workers=4)

In [11]:
ls = []
for epoch in range(50):
    model.train()
    pred_list = []
    label_list = []
    
    loss_list = []
    for seq, label in tqdm(train_loader):
        seq = paddle.reshape(seq, (-1,4,size,size ))
        seq = paddle.cast(seq, dtype='float32')
        label = paddle.cast(label, dtype='int')
        
        pred = model(seq)
        
        loss = criterion(pred.squeeze()/1000, label/1000)
        loss_list.append(loss)
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        pred_list.extend(pred.squeeze().cpu().detach().numpy())
        label_list.extend(label.squeeze().cpu().detach().numpy())
        del pred 
        del seq
        del label
    
    total_loss = paddle.mean(paddle.to_tensor(loss_list))
#     train_score = cal_score(pred_list, label_list)
    model.eval()

    print(
        f'Epoch: {epoch} Loss: {total_loss}'
    )
    

  0%|          | 0/340 [00:00<?, ?it/s]/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:654: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
  0%|          | 0/340 [00:00<?, ?it/s]

Epoch: 0 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.31119299])


  0%|          | 0/340 [00:00<?, ?it/s]

Epoch: 1 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.29853109])


  0%|          | 0/340 [00:00<?, ?it/s]

...

Epoch: 47 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.15507305])


  0%|          | 0/340 [00:00<?, ?it/s]

Epoch: 48 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.15460750])


100%|██████████| 340/340 [00:04<00:00, 83.39it/s] 

Epoch: 49 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.15445520])


In [12]:
model.eval()
layer_state_dict = model.state_dict()
paddle.save(layer_state_dict, "cnn_copy1.pdparams")